In [19]:
# fichier: models/data_utils.py (par exemple)
import pandas as pd
import numpy as np
from pathlib import Path



def load_equity_prices():
    filepath = r"c:\Users\werid\Desktop\Cours 3A\modéle d'evaluation de produits dérivées\Asset pricing\Asset_Pricing_Project\data\cac40_prices.csv"

    df = pd.read_csv(filepath, index_col=0, parse_dates=True)
    return df

def compute_annual_vol(prices, ticker, trading_days=252):
    """
    prices : DataFrame de prix (colonnes = tickers)
    ticker : ticker à utiliser (ex: 'AAPL')
    """
    series = prices[ticker].dropna()
    # rendements log ou simples, ici log :
    log_returns = np.log(series / series.shift(1)).dropna()
    daily_vol = log_returns.std()
    annual_vol = daily_vol * np.sqrt(trading_days)
    return annual_vol


In [21]:
prices = load_equity_prices()
sigma = compute_annual_vol(prices, "^FCHI")
print("Vol annualisée ^FCHI  :", sigma)


Vol annualisée ^FCHI  : 0.14323837467986927


In [22]:
# fichier: models/equity_model.py
import numpy as np
from scipy.stats import norm

def bs_call_price(S, K, T, r, sigma):
    """
    Prix d'un CALL européen par Black-Scholes.
    """
    if T <= 0 or sigma <= 0:
        return max(S - K, 0.0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def bs_put_price(S, K, T, r, sigma):
    """
    Prix d'un PUT européen par Black-Scholes.
    """
    if T <= 0 or sigma <= 0:
        return max(K - S, 0.0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def bs_call_delta(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return 1.0 if S > K else 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return norm.cdf(d1)

def bs_call_gamma(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return norm.pdf(d1) / (S * sigma * np.sqrt(T))

def bs_call_vega(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

def bs_call_theta(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    term1 = - (S * norm.pdf(d1) * sigma) / (2 * np.sqrt(T))
    term2 = - r * K * np.exp(-r * T) * norm.cdf(d2)
    return term1 + term2


In [24]:
# fichier: tests/test_bs_from_data.py
import pandas as pd




# 1. Charger les prix et calculer S et sigma
prices = load_equity_prices()
ticker = "^FCHI"   # à adapter à ce que tu as téléchargé
S = prices[ticker].dropna().iloc[-1]   # dernier prix
sigma = compute_annual_vol(prices, ticker)


T = 1.0  # maturité 1 an
# 3. Fixer un strike (par exemple at-the-money)
 # option at-the-money
K = S 
r = 0.025   # input choisi par l’utilisateur
call = bs_call_price(S, K, T, r, sigma)



# 4. Calculer prix et grecques
call_price = bs_call_price(S, K, T, r, sigma)
put_price  = bs_put_price(S, K, T, r, sigma)
delta = bs_call_delta(S, K, T, r, sigma)
gamma = bs_call_gamma(S, K, T, r, sigma)
vega  = bs_call_vega(S, K, T, r, sigma)
theta = bs_call_theta(S, K, T, r, sigma)

print(f"Sous-jacent : {ticker}")
print(f"S = {S:.2f}, K = {K:.2f}, T = {T}, r = {r:.4f}, sigma = {sigma:.4f}")
print(f"Prix call BS : {call_price:.4f}")
print(f"Prix put  BS : {put_price:.4f}")
print(f"Delta : {delta:.4f}, Gamma : {gamma:.6f}, Vega : {vega:.4f}, Theta : {theta:.4f}")


Sous-jacent : ^FCHI
S = 7959.67, K = 7959.67, T = 1.0, r = 0.0250, sigma = 0.1432
Prix call BS : 553.9168
Prix put  BS : 357.3919
Delta : 0.5972, Gamma : 0.000339, Vega : 3080.6889, Theta : -325.6300
